# Семинар 3. Исправление опечаток

In [1]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
punct = set(punctuation)
from sklearn.metrics import classification_report

Возьмем данные с соревнования Dialog Evaluation 2015 по исправлению опечаток. Данные представляют собой набор предложений (правильное - ошибочное). Задача найти слова с ошибками и заменить их на правильный вариант.

Недостатком тут является то, что не всегда можно правильно сопоставить слова правильного предложения и ошибочного (из-за слов с пропущенным или добавленным пробелом). Из статьи авторов корпуса не очень понятно, как они решали эту проблему, поэтому я просто удалил все такие предложения.

In [2]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [3]:
# Посмотрим на пары предложений
print(bad[1])
print(true[1])

Опофеозом дня для меня сегодня стала фраза услышанная в новостях:
Апофеозом дня для меня сегодня стала фраза услышанная в новостях


In [4]:
# напишем функцию, которая будет сопоставлять слова в правильном и ошибочном варианте
# разобьем предложение по пробелам и удалим пунктуация на границах слов
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

In [5]:
pprint(align_words(true[1], bad[1]))

[('апофеозом', 'опофеозом'),
 ('дня', 'дня'),
 ('для', 'для'),
 ('меня', 'меня'),
 ('сегодня', 'сегодня'),
 ('стала', 'стала'),
 ('фраза', 'фраза'),
 ('услышанная', 'услышанная'),
 ('в', 'в'),
 ('новостях', 'новостях')]


Вытащим только неправильные варианты и заодно посчитаем процент ошибок.

In [6]:
mistakes = []
total = 0
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if pair[0] != pair[1]:
            mistakes.append(pair)
        total += 1

In [7]:
print('Доля ошибок - ', len(mistakes)/total )

Доля ошибок -  0.13034358769476628


Обернем в Counter, чтобы сразу увидеть частотные ошибки.

In [8]:
Counter(mistakes).most_common(10)

[(('сегодня', 'седня'), 24),
 (('вообще', 'вобще'), 18),
 (('вообще', 'ваще'), 17),
 (('естественно', 'естесственно'), 17),
 (('хочется', 'хочеться'), 16),
 (('кстати', 'кстате'), 16),
 (('очень', 'ооочень'), 14),
 (('как-то', 'както'), 9),
 (('очень', 'оооочень'), 9),
 (('это', 'ето'), 9)]

Из-за того, что процент ошибок довольно низкий, не очень выгодно будет находить исправление для каждого слова. Нужен какой-то более простой классификатор, который выделит ошибочные слова, чтобы потом только их и редактировать.

Самый простой способ это сделать - составить словарь правильных слов и потом сравнивать с ним. Чтобы не делать этого вручную, можно взять какой-нибудь корпус текстов, прошедщих редактуру. Новостные тексты для этого хорошо подходят.

Я заранее собрал небольшой корпус новостных текстов и немного почистил его удалив отдельную пунктуацию и пунктуацию на границах слов.

In [9]:
corpus = open('corpus_ng.txt', encoding='utf8').read().splitlines()

In [10]:
# нормализация нам тут не нужна так как нужно находить слова в разных формах
print(corpus[1].split()[:10])

['судя', 'по', 'всему', 'русская', 'православная', 'церковь', 'нашла', 'долгожданную', 'национальную', 'идею']


## Задание 1.
Напишите функцию, которая будет предсказывать ошибочные слова на основе корпуса.

In [11]:
# создайте множество, чтобы проверять вхождения
vocab = set()
# добавляем слова в корпус
for text in corpus:
    vocab.update(text.split())

In [12]:
def predict_mistaken(word, vocab):
    '''
    ::input: word, vocabulary
    ::output: 1 or 0
    
    '''
    if word not in vocab:
        return 1
    else:
        return 0

In [18]:
# для оценки создайте два списка y_true и y_pred
y_true = []
y_pred = []

for i in range(len(bad)):
    # пройдитесь по предложениям, сопоставьте слова с помощью функции align_words
    aligned = align_words(bad[i], true[i])
    # пройдитесь по парам слов и
    for pair in aligned:
        # если слова одинаковые добавьте в y_true 0 
        if pair[0] == pair[1]:
            y_true.append(0)
        # если слова разные добавьте в y_true 1
        else:
            y_true.append(1)
        # предскажите ошибочность слова из bad списка, добавьте предсказание в список y_pred
        y_pred.append(predict_mistaken(pair[0], vocab))

**NB:** для построения `classification_report` нужно иметь два массива одинаковой длины, поэтому мы делаем предсказание для первого слова из `aligned` в любом случае.

In [19]:
# оцените качество с помощью classification_report
print(classification_report(y_true, y_pred))

             precision    recall  f1-score   support

          0       0.98      0.89      0.93      8707
          1       0.55      0.88      0.68      1305

avg / total       0.92      0.89      0.90     10012



### Генерация исправлений

Теперь нужно думать о том, как исправить неправильные слова. Посмотрим как это можно делать на примере известной реализации Питера Норвига.

Основная идея - сделать словарь правильных слов (у нас уже есть), расчитать вероятность каждого слова в корпусе.

In [20]:
corpus = [sent.split() for sent in open('corpus_ng.txt', encoding='utf8').read().splitlines()]
WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)

In [21]:
WORDS.most_common(10)

[('в', 67679),
 ('и', 55933),
 ('на', 27860),
 ('не', 21627),
 ('что', 18299),
 ('с', 18224),
 ('по', 13117),
 ('а', 9696),
 ('как', 8958),
 ('к', 8907)]

In [22]:
# фунцкия расчета вероятности слова
N = sum(WORDS.values())
def P(word, N=N): 
    "Вычисляем вероятность слова"
    return WORDS[word] / N


In [23]:
P('слово')

0.00012798073439310994

Чтобы найти исправления нужно сгенерировать возможные исправления и выбрать те, которые есть в словаре. Если есть несколько вариантов, то выбрать тот, у котогоро наибольшая вероятность.

In [24]:
def correction(word): 
    "Находим наиболее вероятное похожее слово"
    return max(candidates(word), key=P)

def candidates(word): 
    "Генерируем кандидатов на исправление"
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "Выбираем слова, которые есть в корпусе"
    return set(w for w in words if w in WORDS)

def edits1(word):
    "Создаем кандидатов, которые отличаются на одну букву"
    letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "Создаем кандидатов, которые отличаются на две буквы"
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [25]:
%%time
correction('опефеоз')

CPU times: user 214 ms, sys: 8.54 ms, total: 222 ms
Wall time: 225 ms


'апофеоз'

In [ ]:
word = 'опефеоз'
splits = [(word[:i], word[i:])    for i in range(len(word) + 1)]

In [ ]:
splits[:10]

In [ ]:
deletes = [L + R[1:] for L, R in splits if R]

In [ ]:
deletes[:10]

In [ ]:
transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]

In [ ]:
transposes[:10]

In [ ]:
letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
replaces = [L + c + R[1:] for L, R in splits if R for c in letters]

In [ ]:
replaces[:10]

In [ ]:
inserts = [L + c + R for L, R in splits for c in letters]

In [ ]:
inserts[:10]

Для оценки используем просто долю правильных исправлений.

In [26]:
# До этого бы уже считали долю ошибок во всех предложениях.
# Поэтому если ничего не менять то доля правильных исправлений уже будет 100 - 13 = 87 %.
# Наш подход соответственно должен показывать лучший результат 
correct = 0
total = 0
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        predicted = correction(pair[1])
        if predicted == pair[0]:
            correct += 1
        total += 1
    if not i % 10:
        print(i)
        print(correct/total)

0
0.7333333333333333
10
0.8384615384615385
20
0.8441558441558441
30
0.8434343434343434
40
0.847953216374269
50
0.8518518518518519
60
0.8444444444444444
70
0.849009900990099
80
0.8490967056323061
90
0.8516699410609038
100
0.850358422939068
110
0.853904282115869
120
0.8538283062645011
130
0.8527472527472527
140
0.8516746411483254
150
0.8574144486692015
160
0.8567216981132075
170
0.8562605277933745
180
0.8536324786324786
190
0.8547094188376754
200
0.8576238576238576
210
0.8588929219600726
220
0.8594687232219366
230
0.8595482546201232
240
0.8635485117897178
250
0.8615497612926919
260
0.862469222652128
270
0.8600405679513184
280
0.8602775088738303
290
0.8581648522550545
300
0.8590745192307693
310
0.8598240469208212
320
0.8602211511199319
330
0.8588787627727147
340
0.8580280172413793
350
0.8573671813661345
360
0.8573238722757223
370
0.8580997271148598
380
0.8593486127864898
390
0.8600189483657035
400
0.8617490141498492
410
0.8611361587015329
420
0.8609696701350454
430
0.8611708996986656
440


KeyboardInterrupt: 

In [27]:
print(correct/total)

0.8601243339253997


In [28]:
%%time
correction('насмехатьсяаававттававаываываы')

CPU times: user 3.37 s, sys: 21.5 ms, total: 3.39 s
Wall time: 3.42 s


'насмехатьсяаававттававаываываы'

In [29]:
[(wt[0], wt[1], correction(wt[1])) for wt, _ in Counter(mistakes).most_common(10)]

[('сегодня', 'седня', 'сеня'),
 ('вообще', 'вобще', 'вообще'),
 ('вообще', 'ваще', 'ваще'),
 ('естественно', 'естесственно', 'естественно'),
 ('хочется', 'хочеться', 'хочется'),
 ('кстати', 'кстате', 'кстати'),
 ('очень', 'ооочень', 'очень'),
 ('как-то', 'както', 'какао'),
 ('очень', 'оооочень', 'оооочень'),
 ('это', 'ето', 'ето')]

### Метрики близости слов.

Вместо того, чтобы генерировать все варианты, можно искать похожие слова в словаре. Для этого нужно задать метрику похожести. Для исправления опечаток часто используются расстояния редактирования (количество редактирований, которые нужно сделать в строке a, чтобы прийти к b.

In [30]:
# в питоне есть библиотеке для нахождения близких строк
from difflib import get_close_matches

In [31]:
%%time
get_close_matches('опофеоз', WORDS.keys(), n=1)

CPU times: user 1.03 s, sys: 169 ms, total: 1.2 s
Wall time: 1.29 s


['апофеоз']

Работает тоже не очень быстро.

Недавно вышла библиотека, в которой реализованы многие методы нахождения расстояний.

In [32]:
import textdistance

In [33]:
def get_closest_match_with_metric(text, lookup, metric=textdistance.hamming):
    similarities = Counter()
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(1)[0]

In [34]:
%%time
get_closest_match_with_metric('апофиоз', WORDS, textdistance.levenshtein)

CPU times: user 22.7 s, sys: 291 ms, total: 23 s
Wall time: 23.9 s


('апофеоз', 0.8571428571428572)

In [35]:
%%time
get_closest_match_with_metric('апофиоз', WORDS, textdistance.levenshtein)

CPU times: user 21.8 s, sys: 68.6 ms, total: 21.8 s
Wall time: 21.9 s


('апофеоз', 0.8571428571428572)

In [36]:
(37000*len(true))/1000/60

564.8666666666667

Ждать 13 минут мы не можем, поэтому попробуем что-то побыстрее.

Многие вещи, которые медленно решаются в питоне, можно оптимизировать с помощью матричных и векторных операций.

Сделаем поиск похожих по векторам символов, из которых состоит слово.

In [37]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [38]:
corpus = [sent.split() for sent in open('corpus_ng.txt', encoding='utf8').read().splitlines()]
WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)

In [39]:
# тут был чит
for sent in true[:500]:
    for token in sent.lower().split():
        WORDS.update([re.sub('(^\W+|\W+$)', '', token)])

In [40]:
vocab = list(WORDS.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec = TfidfVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

In [41]:
def get_closest_match_vec(text, X, vec, TOPN=3):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)
    topn = similarities.argsort()[0][:TOPN]
    
    return [id2word[top] for top in topn]

In [43]:
%%time
get_closest_match_vec("опофеоз", X, vec)

CPU times: user 45.7 ms, sys: 11.4 ms, total: 57.1 ms
Wall time: 56.6 ms


['апофеоз', 'апофеозом', 'апофеоза']

## Задание 2. 


Напишите функцию, которая принимает слово и находит ближайшее к нему в словаре (сгенерируйте несколько кандидатов с помощью get_closest_match_vec, а затем посчитайте метрики близости до кадого слова и выведете самое близкое). 

In [47]:
def get_closest_hybrid_match(text, X, vec, metric=textdistance.levenshtein):
    # ваш код здесь
    candidates = get_closest_match_vec(text, X, vec, TOPN=3)
    similarities = [metric.normalized_similarity(text, candidate) for candidate in candidates]
    closest = candidates[similarities.index(max(similarities))]
    return closest

In [49]:
# оцените качество также как и раньше
correct = 0
total = 0
for i in range(500, len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        predicted = get_closest_hybrid_match(pair[1], X, vec)
        if predicted == pair[0]:
            correct += 1
        total += 1
    
    if not i % 50:
        print(i)
        print(correct/total)

500
0.7142857142857143
550
0.8038834951456311
600
0.8303249097472925
650
0.83451536643026
700
0.8301382077574677
750
0.8231464737793852
800
0.8213851761846902
850
0.8191815856777493
900
0.8237530753746366
